In [32]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 20)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(20, 5)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = MyModel()

In [34]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /Users/arcmode/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/arcmode/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████████████████████████████████████████████| 44.7M/44.7M [00:00<00:00, 61.6MB/s]


In [40]:
# List to store information about each layer
layers = []

# Hook function to capture input/output shapes and parameters
def hook_fn(module, input, output):
    layer_info = {}
    layer_info['name'] = module.__class__.__name__
    # Input is a tuple; get the shape of the first tensor
    layer_info['input_shape'] = tuple(input[0].shape)
    # Output can be a tensor or tuple
    if isinstance(output, torch.Tensor):
        layer_info['output_shape'] = tuple(output.shape)
    elif isinstance(output, tuple):
        layer_info['output_shape'] = tuple(output[0].shape)
    else:
        layer_info['output_shape'] = None
    # Count parameters
    layer_info['num_parameters'] = sum(p.numel() for p in module.parameters())
    # Iterate over named parameters
    layer_info['parameters'] = {}
    for name, param in module.named_parameters():
        layer_info['parameters'][name] = param.shape
    layers.append(layer_info)

# Attach hooks to layers of interest
hooks = []
for name, module in model.named_modules():
    if isinstance(module, (nn.Linear, nn.Conv2d, nn.ReLU, nn.MaxPool2d)):
        h = module.register_forward_hook(hook_fn)
        hooks.append(h)

# Pass a sample input through the model
sample_input = torch.randn(2, 3, 4, 5)   # Adjust size as needed
model(sample_input)

# Remove hooks after use
for h in hooks:
    h.remove()

In [41]:
from IPython.display import Markdown

def generate_mermaid_code(layers):
    mermaid_code = "graph LR\n"
    for i, layer_info in enumerate(layers):
        layer_label = f"<strong><em>{layer_info['name']}</em></strong><br>" \
                      "----------------<br>" \
                      f"Input: {layer_info['input_shape']}<br>" \
                      f"Output: {layer_info['output_shape']}<br>" \
                      f"Params: {layer_info['num_parameters']}<br>" \
                      "----------------<br>" \
                      + "<br>".join([f"{name}: {list(shape)}" for name, shape in layer_info['parameters'].items()])
        mermaid_code += f"    N{i}[\"{layer_label}\"]\n"
        if i > 0:
            mermaid_code += f"    N{i-1} --> N{i}\n"
    return mermaid_code

mermaid_code = generate_mermaid_code(layers)

Markdown(f"""```mermaid
{mermaid_code}```""")

```mermaid
graph LR
    N0["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N1["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N0 --> N1
    N2["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N1 --> N2
    N3["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N2 --> N3
    N4["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 2, 3)<br>Params: 0<br>----------------<br>"]
    N3 --> N4
    N5["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 2, 3)<br>Params: 0<br>----------------<br>"]
    N4 --> N5
    N6["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 2, 3)<br>Params: 0<br>----------------<br>"]
    N5 --> N6
    N7["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 2, 3)<br>Params: 0<br>----------------<br>"]
    N6 --> N7
    N8["<strong><em>MaxPool2d</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N7 --> N8
    N9["<strong><em>MaxPool2d</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N8 --> N9
    N10["<strong><em>MaxPool2d</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N9 --> N10
    N11["<strong><em>MaxPool2d</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N10 --> N11
    N12["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N11 --> N12
    N13["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N12 --> N13
    N14["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N13 --> N14
    N15["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N14 --> N15
    N16["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N15 --> N16
    N17["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N16 --> N17
    N18["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N17 --> N18
    N19["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N18 --> N19
    N20["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N19 --> N20
    N21["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N20 --> N21
    N22["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N21 --> N22
    N23["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N22 --> N23
    N24["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N23 --> N24
    N25["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N24 --> N25
    N26["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N25 --> N26
    N27["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N26 --> N27
    N28["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N27 --> N28
    N29["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N28 --> N29
    N30["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N29 --> N30
    N31["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N30 --> N31
    N32["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N31 --> N32
    N33["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N32 --> N33
    N34["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N33 --> N34
    N35["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N34 --> N35
    N36["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N35 --> N36
    N37["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N36 --> N37
    N38["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N37 --> N38
    N39["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 36864<br>----------------<br>weight: [64, 64, 3, 3]"]
    N38 --> N39
    N40["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N39 --> N40
    N41["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N40 --> N41
    N42["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N41 --> N42
    N43["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 64, 1, 2)<br>Params: 0<br>----------------<br>"]
    N42 --> N43
    N44["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 73728<br>----------------<br>weight: [128, 64, 3, 3]"]
    N43 --> N44
    N45["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 73728<br>----------------<br>weight: [128, 64, 3, 3]"]
    N44 --> N45
    N46["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 73728<br>----------------<br>weight: [128, 64, 3, 3]"]
    N45 --> N46
    N47["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 73728<br>----------------<br>weight: [128, 64, 3, 3]"]
    N46 --> N47
    N48["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N47 --> N48
    N49["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N48 --> N49
    N50["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N49 --> N50
    N51["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N50 --> N51
    N52["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N51 --> N52
    N53["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N52 --> N53
    N54["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N53 --> N54
    N55["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N54 --> N55
    N56["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 8192<br>----------------<br>weight: [128, 64, 1, 1]"]
    N55 --> N56
    N57["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 8192<br>----------------<br>weight: [128, 64, 1, 1]"]
    N56 --> N57
    N58["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 8192<br>----------------<br>weight: [128, 64, 1, 1]"]
    N57 --> N58
    N59["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 64, 1, 2)<br>Output: (2, 128, 1, 1)<br>Params: 8192<br>----------------<br>weight: [128, 64, 1, 1]"]
    N58 --> N59
    N60["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N59 --> N60
    N61["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N60 --> N61
    N62["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N61 --> N62
    N63["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N62 --> N63
    N64["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N63 --> N64
    N65["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N64 --> N65
    N66["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N65 --> N66
    N67["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N66 --> N67
    N68["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N67 --> N68
    N69["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N68 --> N69
    N70["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N69 --> N70
    N71["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N70 --> N71
    N72["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N71 --> N72
    N73["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N72 --> N73
    N74["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N73 --> N74
    N75["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 147456<br>----------------<br>weight: [128, 128, 3, 3]"]
    N74 --> N75
    N76["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N75 --> N76
    N77["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N76 --> N77
    N78["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N77 --> N78
    N79["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 128, 1, 1)<br>Params: 0<br>----------------<br>"]
    N78 --> N79
    N80["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 294912<br>----------------<br>weight: [256, 128, 3, 3]"]
    N79 --> N80
    N81["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 294912<br>----------------<br>weight: [256, 128, 3, 3]"]
    N80 --> N81
    N82["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 294912<br>----------------<br>weight: [256, 128, 3, 3]"]
    N81 --> N82
    N83["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 294912<br>----------------<br>weight: [256, 128, 3, 3]"]
    N82 --> N83
    N84["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N83 --> N84
    N85["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N84 --> N85
    N86["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N85 --> N86
    N87["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N86 --> N87
    N88["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N87 --> N88
    N89["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N88 --> N89
    N90["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N89 --> N90
    N91["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N90 --> N91
    N92["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 32768<br>----------------<br>weight: [256, 128, 1, 1]"]
    N91 --> N92
    N93["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 32768<br>----------------<br>weight: [256, 128, 1, 1]"]
    N92 --> N93
    N94["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 32768<br>----------------<br>weight: [256, 128, 1, 1]"]
    N93 --> N94
    N95["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 128, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 32768<br>----------------<br>weight: [256, 128, 1, 1]"]
    N94 --> N95
    N96["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N95 --> N96
    N97["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N96 --> N97
    N98["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N97 --> N98
    N99["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N98 --> N99
    N100["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N99 --> N100
    N101["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N100 --> N101
    N102["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N101 --> N102
    N103["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N102 --> N103
    N104["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N103 --> N104
    N105["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N104 --> N105
    N106["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N105 --> N106
    N107["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N106 --> N107
    N108["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N107 --> N108
    N109["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N108 --> N109
    N110["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N109 --> N110
    N111["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 589824<br>----------------<br>weight: [256, 256, 3, 3]"]
    N110 --> N111
    N112["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N111 --> N112
    N113["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N112 --> N113
    N114["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N113 --> N114
    N115["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 256, 1, 1)<br>Params: 0<br>----------------<br>"]
    N114 --> N115
    N116["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 1179648<br>----------------<br>weight: [512, 256, 3, 3]"]
    N115 --> N116
    N117["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 1179648<br>----------------<br>weight: [512, 256, 3, 3]"]
    N116 --> N117
    N118["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 1179648<br>----------------<br>weight: [512, 256, 3, 3]"]
    N117 --> N118
    N119["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 1179648<br>----------------<br>weight: [512, 256, 3, 3]"]
    N118 --> N119
    N120["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N119 --> N120
    N121["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N120 --> N121
    N122["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N121 --> N122
    N123["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N122 --> N123
    N124["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N123 --> N124
    N125["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N124 --> N125
    N126["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N125 --> N126
    N127["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N126 --> N127
    N128["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 131072<br>----------------<br>weight: [512, 256, 1, 1]"]
    N127 --> N128
    N129["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 131072<br>----------------<br>weight: [512, 256, 1, 1]"]
    N128 --> N129
    N130["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 131072<br>----------------<br>weight: [512, 256, 1, 1]"]
    N129 --> N130
    N131["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 256, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 131072<br>----------------<br>weight: [512, 256, 1, 1]"]
    N130 --> N131
    N132["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N131 --> N132
    N133["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N132 --> N133
    N134["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N133 --> N134
    N135["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N134 --> N135
    N136["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N135 --> N136
    N137["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N136 --> N137
    N138["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N137 --> N138
    N139["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N138 --> N139
    N140["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N139 --> N140
    N141["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N140 --> N141
    N142["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N141 --> N142
    N143["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N142 --> N143
    N144["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N143 --> N144
    N145["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N144 --> N145
    N146["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N145 --> N146
    N147["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 2359296<br>----------------<br>weight: [512, 512, 3, 3]"]
    N146 --> N147
    N148["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N147 --> N148
    N149["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N148 --> N149
    N150["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N149 --> N150
    N151["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 512, 1, 1)<br>Output: (2, 512, 1, 1)<br>Params: 0<br>----------------<br>"]
    N150 --> N151
    N152["<strong><em>Linear</em></strong><br>----------------<br>Input: (2, 512)<br>Output: (2, 1000)<br>Params: 513000<br>----------------<br>weight: [1000, 512]<br>bias: [1000]"]
    N151 --> N152
    N153["<strong><em>Linear</em></strong><br>----------------<br>Input: (2, 512)<br>Output: (2, 1000)<br>Params: 513000<br>----------------<br>weight: [1000, 512]<br>bias: [1000]"]
    N152 --> N153
    N154["<strong><em>Linear</em></strong><br>----------------<br>Input: (2, 512)<br>Output: (2, 1000)<br>Params: 513000<br>----------------<br>weight: [1000, 512]<br>bias: [1000]"]
    N153 --> N154
    N155["<strong><em>Linear</em></strong><br>----------------<br>Input: (2, 512)<br>Output: (2, 1000)<br>Params: 513000<br>----------------<br>weight: [1000, 512]<br>bias: [1000]"]
    N154 --> N155
```

In [43]:
print(mermaid_code)

graph LR
    N0["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N1["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N0 --> N1
    N2["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N1 --> N2
    N3["<strong><em>Conv2d</em></strong><br>----------------<br>Input: (2, 3, 4, 5)<br>Output: (2, 64, 2, 3)<br>Params: 9408<br>----------------<br>weight: [64, 3, 7, 7]"]
    N2 --> N3
    N4["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64, 2, 3)<br>Params: 0<br>----------------<br>"]
    N3 --> N4
    N5["<strong><em>ReLU</em></strong><br>----------------<br>Input: (2, 64, 2, 3)<br>Output: (2, 64